---
format: 
  html: default
  typst: default
jupyter: python3
---

# Inversión de ERT con topografía

In [ ]:
import numpy as np
import scipy
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pygimli as pg
import pygimli.meshtools as mt
from pygimli.physics import ert
from pygimli.frameworks import PriorModelling

## Cargamos los datos

Cargamos los datos con `ert.load("filename.dat")` que soporta diferentes tipos de formatos.

In [ ]:
data = ert.load('ERT/datos/DipoleDipole_1.dat', verbose=True)
print(data)
tomocode = "DD-1" # Código del perfil, nombre a usar en archivos de salida

Exploramos la topografía en los datos, si la tiene.

In [ ]:
xpos = pg.x(data)
ypos = pg.y(data)
zpos = pg.z(data)

print(xpos)
print(ypos)
print(zpos)

In [ ]:
lp = max(xpos - min(xpos))  # longitud del perfil
deltaY = max(ypos - min(ypos))
deltaZ = max(zpos - min(zpos))

print(f"Longitud del perfil: {lp:.2f} m")

print(f"Elevación mínima (Y): {min(ypos):.2f} m")
print(f"Elevación máxima (Y): {max(ypos):.2f} m")
print(f"Diferencia de elevación (Y): {deltaY:.2f} m")

print(f"Elevación mínima (Z): {min(zpos):.2f} m")
print(f"Elevación máxima (Z): {max(zpos):.2f} m")
print(f"Diferencia de elevación (Z): {deltaZ:.2f} m")

Si no se tiene incluida la topografía en el archivo o no la reconoce se puede agregar por medio de un archivo (x,y), si se tiene encabezado de columnas se usa `skiprows=1` si no se usa `skiprows=0`. El archivo de topografía tiene que comprender los extremos del perfil para evitar extrapolación.

In [ ]:
topo = np.loadtxt('ERT/datos/DD_topo.txt', skiprows=1)

Se interpolan los datos topográficos a las posiciones de los electrodos (sensores), el argumento `k` de `make_interp_spline` define si la interpolación es lineal (1) o con b-splines (3).

In [ ]:
bspl = make_interp_spline(topo[:,0], topo[:,1], k=3)
z_interp = bspl(xpos)
print(z_interp)

In [ ]:
sensor_positions = data.sensorPositions()
for i in range(len(sensor_positions)):
  pos = sensor_positions[i]
  pos[2] = z_interp[i]
  data.setSensorPosition(i,pos)

Si leyó la topografía pero está en el eje Y en lugar del eje Z se puede corregir de la siguiente forma, esto para que al exportar el archivo a otros formatos la topografía quede en el eje Z. Por defecto no se va a evaluar este bloque. _Después de correr la inversión la topografía va a quedar en el eje Y automáticamente_.

In [ ]:
#| eval: false

sensor_positions = data.sensorPositions()
for i in range(len(sensor_positions)):
  pos = sensor_positions[i]
  pos[2] = pos[1]
  pos[1] = 0.0  # opcional, para dejar Y=0.0 m
  data.setSensorPosition(i,pos)

Visualizamos la topografía.

In [ ]:
plt.figure()
ax = plt.gca()
ax.set_title('Elevación del perfil de tomografía')
ax.set_ylabel('Z (m)')
ax.set_xlabel('Distancia (m)')
ax.set_aspect(aspect=10) # exageracion vertical
# ax.invert_xaxis() # invierte la direccion del eje-x
plt.plot(pg.x(data), pg.z(data), 'x-', color='b')
plt.show()

Salva los datos en un formato unificado.

In [ ]:
data.save('ERT/datos/DD_BERT.dat')

## Calculamos valores necesarios para la inversión

Los datos no traen el factor geométrico ('k'), por lo que tenemos que crearlos.

In [ ]:
k0 = ert.createGeometricFactors(data)  # the analytical one
data['k'] = ert.createGeometricFactors(data, numerical=True)

Calcula la resistividad aparente en caso de que no la lea del archivo.

In [ ]:
if np.std(data['rhoa']) == 0:
  data['rhoa'] = data['r'] * data['k']

Los datos por lo general no traen un campo de errores ('err'), requiriendo que los agreguemos.

In [ ]:
data.estimateError(relativeError=0.03, absoluteUError=5e-5)
# which internally calls
# data['err'] = ert.estimateError(data, ...)  # can also set manually
_ = data.show(data['err']*100, label='Error (%)')

## Pseudosección

In [ ]:
ax, cb = ert.showData(data, 
                      vals=data['rhoa'], 
                      cMap='Spectral_r', 
                      logScale=True
                      )
cb.set_label('Resistividad aparente ($\Omega$m)', 
             fontsize=11, fontweight='normal')

# plt.savefig('ERT/figs/ert-' + tomocode + '-pseudo.png', dpi=300, bbox_inches='tight')

## Filtrado de datos

En caso de requerir filtrar (limpiar) los datos de acuerdo a valores de alguna propiedad se pueden marcar como inválidos, o usar el método `.remove` sobre los datos para removerlos definitivamente.

In [ ]:
data.markInvalid(data['rhoa'] > 100)
data

In [ ]:
data.remove(data['rhoa'] > 100)
data

## Inversión de los datos

Para realizar la inversión se tiene que crear el administrador de la inversión con `ERTManager`.

In [ ]:
mgr_ert = ert.ERTManager(data)

La inversión se puede correr con parámetros por defecto o se pueden ajustar:

-   `lam`: suavizado, mientras más alto mayor suavizado (20 es el defecto)
-   `paraDX`: división de elementos entre sensores (0.5 es 1 nodo adicional, 0.33 son 2)
-   `paraMaxCellSize`: área máxima de los elementos de la malla
-   `paraDepth`: profundidad de la malla
-   `quality`: calidad (refinado) de la malla, valores entre 33 y 35 son recomendados

Adicionalmente se puede ajustar la regularización:

-   `zWeight`: relación entre vertical/horizontal, valores menores a 1 enfatizan estructuras horizontales, mayores a 1 estructuras verticales
-   `cType`: tipo de regularizacion, 1 es el defecto, 2 es mas suavizado
-   `correlationLengths`: lista `[h, v]` indicando la distancia de correlacion horizontal y vertical
-   `limits`: lista `[min max]` indicando los limites de los valores de la variable a invertir

In [ ]:
mgr_ert.inv.setRegularization(cType=1, zWeight=0.5)

In [ ]:
mod = mgr_ert.invert(data, 
                     verbose=True,
                     lam=10, 
                     paraDX=0.3, 
                     paraMaxCellSize=10, 
                     paraDepth=lp*.2, # profundidad al 20% de la longitud del perfil
                    #  paraDepth=35, # profundidad fija
                     quality=34.4
                     )

### Cambio en el $\chi^2$

In [ ]:
chihx = np.arange(1,len(mgr_ert.inv.chi2History)+1)
chihy = mgr_ert.inv.chi2History

In [ ]:
plt.figure()
plt.ylabel('$\chi^2$', fontsize=16, fontweight='bold')
plt.xlabel('Iteración', fontsize=16, fontweight='bold')
plt.tick_params(labelsize=14)
# plt.yscale('log')
# plt.plot(chihx, chihy, 'o-', color='b')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.semilogy(chihx, chihy, 'o-', color='b')
plt.show()

# plt.savefig('ERT/figs/ert-' + tomocode + '-chi2-iter.png', dpi=300, bbox_inches='tight')

### Valores mínimos y máximos

In [ ]:
rmin = min(mgr_ert.model)
rmax = max(mgr_ert.model)

print(f"Rmin: {rmin:.1f} ohm.m")
print(f"Rmax: {rmax:.1f} ohm.m")

In [ ]:
rlist = ["Resistividades del modelo\n", 
         "Mínima: " + str(round(rmin,2)) + " ohm.m\n", 
         "Máxima: " + str(round(rmax,2)) + " ohm.m\n"
         "\n",
         "Estadísticas del ajuste\n",
         f"\u03C7^2: {mgr_ert.inv.chi2():.2f}\n",
         f"Rel RMS: {mgr_ert.inv.relrms():.2f} %\n"]

with open("ERT/resistividades-" + tomocode + "-.txt", "w") as f:
    f.writelines(rlist)

## Visualización de los resultados

In [ ]:
ax, cb = mgr_ert.showResult()

Podemos ver los resultados de diferentes formas: - Resultado y ajuste - Ajuste - Diferencia con el ajuste

In [ ]:
_ = mgr_ert.showResultAndFit()
# np.testing.assert_approx_equal(ert.inv.chi2(), 1.10883, significant=3)

# plt.savefig('ERT/figs/tomo-res-fit.png', dpi=300, bbox_inches='tight')

In [ ]:
mgr_ert.showFit()

# plt.savefig('ERT/figs/tomo-fit.png', dpi=300, bbox_inches='tight')

In [ ]:
mgr_ert.showMisfit()

# plt.savefig('ERT/figs/tomo-misfit.png', dpi=300, bbox_inches='tight')

### Errores entre observados y modelados

In [ ]:
obsRES = np.array(mgr_ert.data('rhoa'))
modRES = np.array(mgr_ert.inv.response)
resRES = obsRES - modRES
r, p = scipy.stats.pearsonr(obsRES, modRES)

print(f"r: {r:.3f}")
print(f"p: {p:.3f}")

In [ ]:
plt.figure(figsize=(12, 8))

ax = plt.subplot(1,2,1)
ax.scatter(obsRES,modRES,alpha=.2,s=20)
ax.axline((0,0),slope=1,color='k',ls='--',lw=.5)
ax.set_xlabel('Resistividades observadas ($\Omega$m)', fontsize=14)
ax.set_ylabel('Resistividades modeladas ($\Omega$m)', fontsize=14)
ax.tick_params(labelsize=12)

plt.subplot(1,2,2)
plt.hist(resRES,edgecolor='k')
plt.xlabel('Residuos (Obs - Mod) ($\Omega$m)', fontsize=14)
plt.tick_params(labelsize=12)

plt.show()

# plt.savefig('ERT/figs/res-fit-err-' + tomocode + '-.png', dpi=300, bbox_inches='tight')

## Modelo (tomografía)

Opciones gráficas para mostrar los resultados.

In [ ]:
kw_ert = dict(cMin=60, 
              cMax=900, 
              cMap="Spectral_r", 
              logScale=True 
              # ,coverage=1
              )

In [ ]:
# ax, cb = pg.show(mgr_ert.paraDomain, mgr_ert.model, nLevs=8, **kw_ert, figsize=(8,4), coverage=mgr_ert.coverage())
ax, cb = mgr_ert.showResult(nLevs=8, **kw_ert)
# ax.set_title(f"Rel RMS: {mgr_ert.inv.relrms():.2f} %")
ax.set_title('Rel RMS={:.2f} %, $\chi^2$={:.2f}'.format(mgr_ert.inv.relrms(), mgr_ert.inv.chi2()))
ax.set_ylabel('Z (m)', fontsize=11, fontweight='normal')
ax.set_xlabel('Distancia (m)', fontsize=11, fontweight='normal')
ax.set_aspect(aspect=1)
ax.set_frame_on(False)
ax.tick_params(labelsize=11)
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)
# ax.spines['bottom'].set_visible(False)
# ax.spines['left'].set_visible(False)
cb.set_label('Resistividad ($\Omega$m)', fontsize=11, fontweight='normal')

print(f"\u03C7^2: {mgr_ert.inv.chi2():.2f}")
print(f"Rel RMS: {mgr_ert.inv.relrms():.2f}%")

# plt.savefig('ERT/figs/tomo-res-' + tomocode + '-.png', dpi=300, bbox_inches='tight')

## Modelo como contornos

Con el argumento `nLevs` se definen la cantidad de clases más 1 que se quieren obtener y con `nCols` se define la cantidad de clases.

### Opción 1 (menos suavizada)

In [ ]:
# ax, cb = pg.show(mgr_ert.paraDomain, mgr_ert.model, nCols=11, nLevs=12, contourLines=False, **kw_ert, figsize=(8,4), coverage=mgr_ert.coverage())
ax, cb = mgr_ert.showResult(nCols=5, nLevs=6, **kw_ert)
# ax.set_title(f"Rel RMS: {mgr_ert.inv.relrms():.2f} %")
ax.set_title('Rel RMS={:.2f} %, $\chi^2$={:.2f}'.format(mgr_ert.inv.relrms(), mgr_ert.inv.chi2()))
ax.set_ylabel('Z (m)')
ax.set_xlabel('Distancia (m)')
# ax.set_aspect(aspect=1)
ax.set_frame_on(False)
cb.set_label('Resistividad ($\Omega$m)')

# plt.savefig('ERT/figs/tomo-res-contour-' + tomocode + '-.png', dpi=300, bbox_inches='tight')

### Opción 2 (más suavizada)

Es necesario convertir los datos de celdas a puntos.

In [ ]:
res_points = pg.core.cellDataToPointData(mgr_ert.paraDomain, mgr_ert.model)

In [ ]:
ax, cbar = pg.show(mgr_ert.paraDomain, res_points, 
                   nLevs=8,
                   cMin=rmin, cMax=rmax, 
                   cMap="RdYlBu_r",
                   logScale=True,
                   linewidths=.5
                   )
pg.viewer.mpl.drawSensors(ax, data.sensorPositions(), 
                          sm='o', diam=2, color='m', alpha=.5)
ax.set_title('Rel RMS={:.2f} %, $\chi^2$={:.2f}'.format(mgr_ert.inv.relrms(), mgr_ert.inv.chi2()))
ax.set_ylabel('Z (m)')
ax.set_xlabel('Distancia (m)')
# ax.set_aspect(aspect=1)
cbar.set_label('Resistividad ($\Omega$m)')

# plt.savefig('ERT/figs/tomo-res-contour-' + tomocode + '-.png', dpi=300, bbox_inches='tight')

In [ ]:
ax, cbar = mgr_ert.showResult(**kw_ert)
pg.viewer.mpl.drawField(ax, mgr_ert.paraDomain, res_points,
                        nLevs=6,
                        # levels = [rmin,...,rmax],
                        cMin=rmin, cMax=rmax, 
                        logScale=True, 
                        fillContour=False, 
                        colors='k',
                        # colors=["c","r","b","g","y"], 
                        linewidths=1, 
                        alpha=0.8
                        )

## Obtener conductividad

In [ ]:
result = mgr_ert.paraDomain
result["res"] = mgr_ert.model
result["cond"] = 1 / result["res"] * 1000

In [ ]:
ax, cb = pg.show(result, result["cond"], cMap="RdYlBu", logScale=True, nLevs=10, figsize=(8,4), coverage=mgr_ert.coverage())
pg.viewer.mpl.drawSensors(ax, data.sensorPositions(), 
                          sm='o', diam=3, color='m', alpha=.5)
ax.set_ylabel('Z (m)')
ax.set_xlabel('Distancia (m)')
# ax.set_aspect(aspect=1)
ax.set_frame_on(False)
cb.set_label('Conductividad (mS/m)')

# plt.savefig('ERT/figs/tomo-cond-' + tomocode + '-.png', dpi=300, bbox_inches='tight')

## Perfil 1D en el centro de la tomografia

Por defecto se toma el centro del perfil, pero se puede cambiar la posición en `pos1D`.

In [ ]:
pos1D = max(pg.x(data))/2  # posición del perfil 1D
reps = 50 # número de puntos en el perfil

bspl2 = make_interp_spline(pg.x(data), pg.y(data), k=3)
y_i = bspl2(pos1D)
y_f = mgr_ert.paraDomain.ymin() # profundidad final del perfil 1D escogida automáticamente
# y_f = -31.0  # profundidad final del perfil 1D escogida manualmente

x = np.repeat(pos1D, reps)
y = np.linspace(y_i, y_f, reps)

In [ ]:
posVec = [pg.Pos(pos) for pos in zip(x, y)]
para = pg.Mesh(mgr_ert.paraDomain)  # make a copy
para.setCellMarkers(pg.IVector(para.cellCount()))
fopDP = PriorModelling(para, posVec)

In [ ]:
fig, ax = plt.subplots(figsize=(4,5))
resSmooth = fopDP(result['res']) # result['cond'] para conductividad
# ax.plot(resSmooth, y, color='orange', label="ERT")
ax.semilogx(resSmooth, y, color='orange', label="ERT")
ax.set_xlabel(r"$\rho$ ($\Omega$m)")
ax.tick_params(axis='x', which='minor', labelsize='small', labelrotation=10)
ax.set_ylabel("Z (m)")
ax.set_title("Perfil 1D" + f" - x={pos1D} m")
ax.grid(True)

# plt.savefig('ERT/figs/tomo-res-' + tomocode + '-1D-' + f"{pos1D} m" + ".png", dpi=300, bbox_inches='tight')

## Salvar resultados

Salva todo

In [ ]:
mgr_ert.saveResult('ERT/tomo-res-' + tomocode + '/')